In [1]:
import numpy as np
E=210000000
v=0.3
t=0.025
w=3000

# Number of nodes
number_of_nodes = 5
number_of_elements = 4

# Enter all x in sequence
x = [0,0.5,0.5,0,0.25]
# Enter all y in sequence
y = [0,0,0.25,0.25,0.125]
connected_nodes = [(1,2,5),(1,5,4),(2,3,5),(3,4,5)]

D = (E/(1-v**2)) * (np.array([1,v,0,v,1,0,0,0,(1-v)/2]).reshape((3,3)))

# Element Stiffness Matrix
areas = {}
B_mat = {}
k_mat = {}
for i in range(number_of_elements):
  a,b,c = connected_nodes[i]
  #Area of elements
  areas[i] = (x[a-1]*(y[b-1]-y[c-1])+x[b-1]*(y[c-1]-y[a-1])+x[c-1]*(y[a-1]-y[b-1]))/2
  #B Matrix Calculation
  B_mat[i] = (np.array([y[b-1]-y[c-1],0,y[c-1]-y[a-1],0,y[a-1]-y[b-1],0,0,x[c-1]-x[b-1],0,x[a-1]-x[c-1],0,x[b-1]-x[a-1],x[c-1]-x[b-1],y[b-1]-y[c-1],x[a-1]-x[c-1],y[c-1]-y[a-1],x[b-1]-x[a-1],y[a-1]-y[b-1]]).reshape((3,6)))/(2*areas[i])
  #k matrix
  k_mat[i] = np.matmul(np.matmul(t*areas[i]*B_mat[i].transpose(),D),B_mat[i])

#TSM
tsm = np.zeros((2*number_of_nodes,2*number_of_nodes))
row_adder=[(-1,0),(0,1),(1,2),(2,3),(3,4)]
for row in range(number_of_elements):
  a,b,c = connected_nodes[row]
  m,n = k_mat[row].shape

  tsm[a+row_adder[a-1][0],[a+row_adder[a-1][0],a+row_adder[a-1][1],b+row_adder[b-1][0],b+row_adder[b-1][1],c+row_adder[c-1][0],c+row_adder[c-1][1]]] += k_mat[row][0,0:m]
  tsm[a+row_adder[a-1][1],[a+row_adder[a-1][0],a+row_adder[a-1][1],b+row_adder[b-1][0],b+row_adder[b-1][1],c+row_adder[c-1][0],c+row_adder[c-1][1]]] += k_mat[row][1,0:m]
  tsm[b+row_adder[b-1][0],[a+row_adder[a-1][0],a+row_adder[a-1][1],b+row_adder[b-1][0],b+row_adder[b-1][1],c+row_adder[c-1][0],c+row_adder[c-1][1]]] += k_mat[row][2,0:m]
  tsm[b+row_adder[b-1][1],[a+row_adder[a-1][0],a+row_adder[a-1][1],b+row_adder[b-1][0],b+row_adder[b-1][1],c+row_adder[c-1][0],c+row_adder[c-1][1]]] += k_mat[row][3,0:m]
  tsm[c+row_adder[c-1][0],[a+row_adder[a-1][0],a+row_adder[a-1][1],b+row_adder[b-1][0],b+row_adder[b-1][1],c+row_adder[c-1][0],c+row_adder[c-1][1]]] += k_mat[row][4,0:m]
  tsm[c+row_adder[c-1][1],[a+row_adder[a-1][0],a+row_adder[a-1][1],b+row_adder[b-1][0],b+row_adder[b-1][1],c+row_adder[c-1][0],c+row_adder[c-1][1]]] += k_mat[row][5,0:m]

print('Total Stiffness Matrix: \n', np.round(tsm,2))

u=[0,0,1,1,1,1,0,0,1,1]
F = np.array([9.375,0,9.375,0,0,0]).reshape((6,1))

count = 0
for i in u:
  if i==1:
    count+=1

reduced_m = []
for i in range(len(u)):
  for j in range(len(u)):
    if u[i]*u[j] == 1:
      reduced_m.append(tsm[i][j])
reduced_m = np.array(reduced_m).reshape((count,count))

disp = np.linalg.solve(reduced_m, F)
print('Displacement: \n', disp)

a=0
b=0
disp_m=[]
for i in u:
  if i == 1:
    disp_m.append(disp[a][0])
    a+=1
  else:
    disp_m.append(u[b])
  b+=1
disp_m = np.array(disp_m).reshape((10,1))

reactions=np.matmul(tsm, disp_m)
print("Reactions:\n",reactions)

u_mat={}
for i in range(len(connected_nodes)):
  a,b,c = connected_nodes[i]
  u_mat[i] = np.array([disp_m[a+row_adder[a-1][0],0],disp_m[a+row_adder[a-1][1],0],disp_m[b+row_adder[b-1][0],0],disp_m[b+row_adder[b-1][1],0],disp_m[c+row_adder[c-1][0],0],disp_m[c+row_adder[c-1][1],0]]).reshape((6,1))

sigma={}
for i in range(len(connected_nodes)):
  sigma[i] = np.matmul(np.matmul(D,B_mat[i]),u_mat[i])
  print('sigma for element '+str(i+1)+': \n', sigma[i])

Total Stiffness Matrix: 
 [[ 3461538.46  1875000.     288461.54   -72115.38        0.          0.
   -288461.54    72115.38 -3461538.46 -1875000.  ]
 [ 1875000.    6274038.46    72115.38  2632211.54        0.          0.
    -72115.38 -2632211.54 -1875000.   -6274038.46]
 [  288461.54    72115.38  3461538.46 -1875000.    -288461.54   -72115.38
         0.          0.   -3461538.46  1875000.  ]
 [  -72115.38  2632211.54 -1875000.    6274038.46    72115.38 -2632211.54
         0.          0.    1875000.   -6274038.46]
 [       0.          0.    -288461.54    72115.38  3461538.46  1875000.
    288461.54   -72115.38 -3461538.46 -1875000.  ]
 [       0.          0.     -72115.38 -2632211.54  1875000.    6274038.46
     72115.38  2632211.54 -1875000.   -6274038.46]
 [ -288461.54   -72115.38        0.          0.     288461.54    72115.38
   3461538.46 -1875000.   -3461538.46  1875000.  ]
 [   72115.38 -2632211.54        0.          0.     -72115.38  2632211.54
  -1875000.    6274038.46  1875